In [3]:
import sys
sys.path.append('C:\\projects\\UTNCE\\src\\utnce202402')

In [4]:
from simplify import *
from prepare import *
from routing import *
from percolation_plot import *
from generate import *
from transfer import *
from copy import deepcopy
from shapely.geometry import Point,LineString
from pyproj import Proj, Transformer
import pickle
import itertools
import random
from sympy import symbols, Eq, solve
import datetime

In [5]:
transitlines = gpd.read_file("C:\\Users\\sga226\\Downloads\\Flow_datasets_GVB\\VENOM2020_pt\\VENOM2020_transitlines.shp")
transitlines

,TRANSITLNR,NAME,MODE,NAMENR,OV_SYSTEME,VERVOERDER,TRANSITTAG,FARENR,PUBLIJNNR,SYSTEEMLIJ,...,PASSTIME_9,REVENUE_9,TRAVTIAB_7,TRAVTIBA_7,TRAVTIAB_8,TRAVTIBA_8,TRAVTIAB_9,TRAVTIBA_9,LIJNKLEURI,geometry
0,16627,ASD_GVB_0050_Isolatorweg>Gein,43,20350,2,1.0,Gein,2,50.0,m50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((123850 482510, 124170 481980..."
1,16628,ASD_GVB_0050_Isolatorweg>Gein,43,19742,2,1.0,Gein,2,50.0,m50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((118469.658 489856.458, 11816..."
2,16629,ASD_GVB_0050_Isolatorweg>Gein,43,20389,2,1.0,Gein,2,50.0,m50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((118469.658 489856.458, 11816..."
3,16630,ASD_GVB_0050_Gein>Isolatorweg,43,19525,2,1.0,Isolatorweg,2,50.0,m50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((127820 478770, 126840 478700..."
4,16631,ASD_GVB_0050_Gein>Isolatorweg,43,19506,2,1.0,Isolatorweg,2,50.0,m50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((127820 478770, 126840 478700..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1547,18462,IJS_CXX_0248_harderwijk - busstation>Station N...,41,21094,4,2.0,Lelystad,2,248.0,m248,...,10.192731,0.0,15.779857,30.462908,15.779857,30.462908,15.779857,30.462908,NaN,"MULTILINESTRING ((170981 483392, 170966 483435..."
1548,18463,LLS_ARR_0777_Lelystad - Winkelcentrum Kempenaa...,41,21097,4,3.0,Airport via Station,2,777.0,m777,...,24.633146,0.0,4.599689,5.061305,4.599689,5.061305,4.599689,5.061305,NaN,"MULTILINESTRING ((160885.5 502180.5, 160885 50..."
1549,18464,LLS_ARR_0777_Lelystad - Airport Plaza>Station ...,41,21098,4,3.0,Station Centrum,2,777.0,m777,...,46.287003,0.0,5.727773,6.414923,5.727773,6.414923,5.727773,6.414923,NaN,"MULTILINESTRING ((164149.012 497107.125, 16439..."
1550,19243,"ONB_ONB_300_Bijlmer ArenA>Haarlem, Station",41,20997,4,2.0,Haarlem Station via Schiphol Airport,2,300.0,m300,...,556.085510,0.0,27.653976,39.223957,27.659529,39.218403,28.374681,39.503250,NaN,"MULTILINESTRING ((124994 480403, 124904 480353..."


In [6]:
transitlines_asd = transitlines[transitlines.CONCESSIE == 'asd_gvb']
transitlines_asd

,TRANSITLNR,NAME,MODE,NAMENR,OV_SYSTEME,VERVOERDER,TRANSITTAG,FARENR,PUBLIJNNR,SYSTEEMLIJ,...,PASSTIME_9,REVENUE_9,TRAVTIAB_7,TRAVTIBA_7,TRAVTIAB_8,TRAVTIBA_8,TRAVTIAB_9,TRAVTIBA_9,LIJNKLEURI,geometry
0,16627,ASD_GVB_0050_Isolatorweg>Gein,43,20350,2,1.0,Gein,2,50.0,m50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((123850 482510, 124170 481980..."
1,16628,ASD_GVB_0050_Isolatorweg>Gein,43,19742,2,1.0,Gein,2,50.0,m50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((118469.658 489856.458, 11816..."
2,16629,ASD_GVB_0050_Isolatorweg>Gein,43,20389,2,1.0,Gein,2,50.0,m50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((118469.658 489856.458, 11816..."
3,16630,ASD_GVB_0050_Gein>Isolatorweg,43,19525,2,1.0,Isolatorweg,2,50.0,m50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((127820 478770, 126840 478700..."
4,16631,ASD_GVB_0050_Gein>Isolatorweg,43,19506,2,1.0,Isolatorweg,2,50.0,m50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((127820 478770, 126840 478700..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1525,18438,ASD_GVB_0026_IJburg>Centraal Station,42,19314,3,1.0,Centraal Station,2,26.0,m26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((131046.434 485876, 131050.23..."
1526,18439,ASD_GVB_0026_Centraal Station>IJburg,42,19450,3,1.0,IJburg,2,26.0,m26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((126546.356 486863.688, 12662..."
1527,18440,ASD_GVB_0026_Centraal Station>IJburg,42,19945,3,1.0,IJburg,2,26.0,m26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((121967 487835, 122023 487826..."
1528,18441,ASD_GVB_0069_Martinair>Station Sloterdijk,41,20538,4,1.0,Station Sloterdijk,2,69.0,m69,...,328.211121,0.0,12.714973,9.881072,14.026931,11.501615,13.494493,11.290913,NaN,"MULTILINESTRING ((112309 480008, 112441 480091..."


In [7]:
gpd.GeoDataFrame(transitlines_asd.copy()).to_file('transitlines_asd0726.gpkg',driver = 'GPKG')